### Tugas Besar B - IF3270 Pembelajaran Mesin
Authors:

13520016 Gagas Praharsa Bahar

13520065 Rayhan Kinan Muhannad

13520081 Andhika Arta Aryanto

13520101 Aira Thalca Avila Putra

#### Install Library




In [4]:
!pip install numpy
!pip install typing
!pip install sklearn

#### Import Library

In [5]:
from __future__ import annotations
import numpy as np
import time
from typing import NamedTuple, Callable
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


### Neural Network

#### Class Row

In [6]:
class Row(NamedTuple):
    weight: np.ndarray

#### Class Activation Function

In [8]:
class ActivationFunction(NamedTuple):
    function: Callable[[np.ndarray], np.ndarray]
    derivative_output: Callable[[np.ndarray, np.ndarray], np.ndarray]

    def __call__(self, o: np.ndarray) -> np.ndarray:
        return self.function(o)

    def get_derivative_output(self, o: np.ndarray, t: np.ndarray) -> np.ndarray:
        return self.derivative_output(o, t)


class LinearActivationFunction(ActivationFunction):
    def __new__(cls) -> LinearActivationFunction:
        # Inner Function
        def linear(x: np.ndarray) -> np.ndarray:
            return np.array(np.vectorize(lambda x: x)(x))

        # Inner Function
        def derivative_output_linear(o: np.ndarray, _: np.ndarray) -> np.ndarray:
            return np.array(np.vectorize(lambda _: 1)(o))

        self = super(LinearActivationFunction, cls).__new__(
            cls,
            linear,
            derivative_output_linear,
        )

        return self


class ReLUActivationFunction(ActivationFunction):
    def __new__(cls) -> ReLUActivationFunction:
        # Inner Function
        def relu(x: np.ndarray) -> np.ndarray:
            return np.array(np.vectorize(lambda x: max(0, x))(x))

        # Inner Function
        def derivative_output_relu(o: np.ndarray, _: np.ndarray) -> np.ndarray:
            return np.array(np.vectorize(lambda o: 1 if o > 0 else 0)(o))

        self = super(ReLUActivationFunction, cls).__new__(
            cls,
            relu,
            derivative_output_relu,
        )

        return self


class SigmoidActivationFunction(ActivationFunction):
    def __new__(cls) -> SigmoidActivationFunction:
        # Inner Function
        def sigmoid(x: np.ndarray) -> np.ndarray:
            return np.array(np.vectorize(lambda x: 1 / (1 + np.exp(-x)))(x))

        # Inner Function
        def derivative_output_sigmoid(o: np.ndarray, _: np.ndarray) -> np.ndarray:
            return np.array(np.vectorize(lambda o: o * (1 - o))(o))

        self = super(SigmoidActivationFunction, cls).__new__(
            cls,
            sigmoid,
            derivative_output_sigmoid,
        )

        return self


class SoftmaxActivationFunction(ActivationFunction):
    def __new__(cls) -> SoftmaxActivationFunction:
        # Inner Function
        def softmax(x: np.ndarray) -> np.ndarray:
            shift_x = np.array(x - np.max(x))  # Normalized
            exps = np.exp(shift_x)
            sums = np.sum(exps, axis=1, keepdims=True)
            return np.array(exps / sums)

        def derivative_output_softmax(o: np.ndarray, t: np.ndarray) -> np.ndarray:
            return np.array(np.subtract(o, t))

        self = super(SoftmaxActivationFunction, cls).__new__(
            cls,
            softmax,
            derivative_output_softmax,
        )

        return self

#### Class Layer

In [9]:
class Layer(NamedTuple):
    list_of_row: list[Row]
    activation_function: ActivationFunction

    def get_output(self, x: np.ndarray) -> np.ndarray:
        array_of_weight = np.array(
            [row.weight for row in self.list_of_row]
        )
        weighted_x = np.array(np.dot(array_of_weight.T, x))
        activated_x = self.activation_function(weighted_x)

        return activated_x

    def get_batch_output(self, batch_x: np.ndarray) -> np.ndarray:
        array_of_weight = np.array(
            [row.weight for row in self.list_of_row]
        )
        weighted_batch_x = np.array(np.dot(batch_x, array_of_weight))
        activated_batch_x = self.activation_function(weighted_batch_x)

        return activated_batch_x

    def get_weight(self) -> np.ndarray:
        return np.array(
            [row.weight for row in self.list_of_row]
        )

    def get_updated_weight(self, delta_weight: np.ndarray) -> Layer:
        array_of_weight = np.array(
            [row.weight for row in self.list_of_row]
        )

        new_array_of_weight = np.add(array_of_weight, delta_weight)
        list_of_row: list[Row] = [
            Row(weight) for weight in new_array_of_weight
        ]

        return Layer(list_of_row, self.activation_function)

#### Class Neural Network

In [10]:
class NeuralNetwork(NamedTuple):
    list_of_layer: list[Layer]

    def get_output(self, x: np.ndarray) -> np.ndarray:
        x_copy: np.ndarray = x.copy()

        for layer in self.list_of_layer:
            bias = np.ones(1)
            biased_x_copy = np.insert(x_copy, 0, bias, axis=0)
            x_copy = layer.get_output(biased_x_copy)

        return x_copy

    def get_batch_output(self, batch_x: np.ndarray) -> np.ndarray:
        N = batch_x.shape[0]
        batch_x_copy: np.ndarray = batch_x.copy()

        for layer in self.list_of_layer:
            bias = np.ones(N)
            biased_batch_x_copy = np.array(np.c_[bias, batch_x_copy])
            batch_x_copy = layer.get_batch_output(biased_batch_x_copy)

        return batch_x_copy

    def get_all_output(self, x: np.ndarray) -> list[np.ndarray]:
        x_copy: np.ndarray = x.copy()
        all_output: list[np.ndarray] = []

        for layer in self.list_of_layer:
            bias = np.ones(1)
            biased_x_copy = np.insert(x_copy, 0, bias, axis=0)
            x_copy = layer.get_output(biased_x_copy)
            all_output.append(x_copy)

        return all_output

    def get_all_batch_output(self, batch_x: np.ndarray) -> list[np.ndarray]:
        N = batch_x.shape[0]
        batch_x_copy: np.ndarray = batch_x.copy()
        all_output: list[np.ndarray] = []

        for layer in self.list_of_layer:
            bias = np.ones(N)
            biased_batch_x_copy = np.array(np.c_[bias, batch_x_copy])
            batch_x_copy = layer.get_batch_output(biased_batch_x_copy)
            all_output.append(batch_x_copy)

        return all_output

    def get_weight(self) -> list[np.ndarray]:
        return [layer.get_weight() for layer in self.list_of_layer]


### Backpropagation

#### Class Error Function

In [11]:
class ErrorFunction(NamedTuple):
    function: Callable[[np.ndarray, np.ndarray], float]

    def __call__(self, o: np.ndarray, t: np.ndarray) -> float:
        return self.function(o, t)


class SumOfSquaredErrorFunction(ErrorFunction):
    def __new__(cls) -> SumOfSquaredErrorFunction:
        # Inner Function
        def sum_of_squared_error(o: np.ndarray, t: np.ndarray) -> float:
            return np.sum(np.sum(np.square(o - t), axis=1) / 2, axis=0)

        self = super(SumOfSquaredErrorFunction, cls).__new__(
            cls,
            sum_of_squared_error,
        )

        return self


class CrossEntropyErrorFunction(ErrorFunction):
    def __new__(cls) -> CrossEntropyErrorFunction:
        # Inner Function
        def cross_entropy_error(o: np.ndarray, t: np.ndarray) -> float:
            return np.sum(-np.sum(t * np.log(o), axis=1), axis=0)

        self = super(CrossEntropyErrorFunction, cls).__new__(
            cls,
            cross_entropy_error,
        )

        return self

#### Class Mini Batch

In [12]:
class MiniBatch(NamedTuple):
    neural_network: NeuralNetwork
    partitioned_learning_data: np.ndarray
    partitioned_learning_target: np.ndarray

    def learn(self, learning_rate: float) -> NeuralNetwork:
        list_of_output = self.neural_network.get_all_batch_output(
            self.partitioned_learning_data
        )
        result: NeuralNetwork = self.neural_network
        previous_delta_error: np.ndarray = None
        new_layer: list[Layer] = [
            None for _ in range(len(list_of_output))
        ]

        for i in range(len(list_of_output) - 1, -1, -1):
            raw_x = self.partitioned_learning_data if i == 0 else list_of_output[i - 1]
            x = np.array(np.c_[np.ones(raw_x.shape[0]), raw_x])

            o = list_of_output[i]
            t = self.partitioned_learning_target
            derivated_output = result.list_of_layer[i].activation_function.get_derivative_output(
                o, t
            )

            # Output Layer
            if i == len(list_of_output) - 1:
                delta_error = -np.multiply(
                    np.subtract(o, t),
                    derivated_output
                ) if type(result.list_of_layer[i].activation_function) is not SoftmaxActivationFunction else -derivated_output
                delta_weight = np.array(
                    np.dot(learning_rate, np.dot(x.T, delta_error))
                )
                new_layer[i] = result.list_of_layer[i].get_updated_weight(
                    delta_weight
                )
                previous_delta_error = delta_error

            # Hidden Layer
            else:
                output_weight = result.list_of_layer[i + 1].get_weight()[1:]
                delta_error = np.multiply(
                    np.dot(
                        previous_delta_error, output_weight.T
                    ),
                    derivated_output
                )
                delta_weight = np.array(
                    np.dot(learning_rate, np.dot(x.T, delta_error))
                )
                new_layer[i] = result.list_of_layer[i].get_updated_weight(
                    delta_weight
                )
                previous_delta_error = delta_error

        for i in range(len(new_layer)):
            result.list_of_layer[i] = new_layer[i]

        return result

#### Class Backpropagation

In [13]:
class Backpropagation(NamedTuple):
    neural_network: NeuralNetwork
    learning_data: np.ndarray
    learning_target: np.ndarray

    def learn(self, learning_rate: float, mini_batch_size: int, max_iter: int, threshold: float, error_function: ErrorFunction) -> NeuralNetwork:
        data_length = self.learning_data.shape[0]
        result: NeuralNetwork = self.neural_network
        current_error = np.inf
        index = 0

        # Until the error is less than or equal to the threshold or the maximum iteration is reached
        while current_error > threshold and index < max_iter:
            start_time = time.time()

            # Get Output
            current_output = result.get_batch_output(self.learning_data)

            # Mini-Batch Learning
            start_index = 0
            while start_index < data_length:
                end_index = min(start_index + mini_batch_size, data_length)
                partitioned_learning_data = self.learning_data[start_index:end_index]
                partitioned_learning_target = self.learning_target[start_index:end_index]

                mini_batch = MiniBatch(
                    result,
                    partitioned_learning_data,
                    partitioned_learning_target,
                )
                result = mini_batch.learn(learning_rate)
                start_index += mini_batch_size

            # Get Error
            current_error = error_function(
                current_output,
                self.learning_target
            )

            finish_time = time.time()

            print(
                f"Epoch {index + 1}\t|\tError: {round(current_error, 4)}\t|\tTime: {round(1000 * (finish_time - start_time), 4)} ms"
            )

            index += 1

        return result

### File System

In [14]:
class FileSystem:
    @staticmethod
    def load_from_file(path: str) -> NeuralNetwork:
        with open(path, "r") as file:
            input_size = int(file.readline().strip())
            num_of_layers = int(file.readline().strip())

            prev_num_of_perceptrons = input_size
            list_of_layer: list[Layer] = []
            latest_activation_function_type: str = None

            for _ in range(num_of_layers):
                num_of_perceptrons = int(file.readline().strip())
                list_of_weight_row: list[Row] = []

                for _ in range(prev_num_of_perceptrons + 1):
                    weight = np.array(
                        list(map(float, file.readline().strip().split()))
                    )
                    row = Row(weight)
                    list_of_weight_row.append(row)

                activation_function_type = file.readline().strip()

                if latest_activation_function_type == "softmax":
                    raise NotImplementedError()

                activation_function: ActivationFunction
                if activation_function_type == "linear":
                    activation_function = LinearActivationFunction()
                elif activation_function_type == "relu":
                    activation_function = ReLUActivationFunction()
                elif activation_function_type == "sigmoid":
                    activation_function = SigmoidActivationFunction()
                elif activation_function_type == "softmax":
                    activation_function = SoftmaxActivationFunction()
                else:
                    raise NotImplementedError()

                layer = Layer(list_of_weight_row, activation_function)
                list_of_layer.append(layer)

                prev_num_of_perceptrons = num_of_perceptrons
                latest_activation_function_type = activation_function_type

            neural_network = NeuralNetwork(list_of_layer)

            return neural_network

    @staticmethod
    def save_to_file(neural_network: NeuralNetwork, path: str) -> None:
        with open(path, "w") as file:
            input_weight = neural_network.list_of_layer[0].get_weight()
            input_size = input_weight.shape[0] - 1
            file.write(f"{input_size}\n")

            num_of_layers = len(neural_network.list_of_layer)
            file.write(f"{num_of_layers}\n")

            for layer in neural_network.list_of_layer:
                weight = layer.get_weight()
                num_of_perceptrons = weight.shape[1]
                file.write(f"{num_of_perceptrons}\n")

                for i in range(weight.shape[0]):
                    for j in range(weight.shape[1]):
                        file.write(f"{weight[i][j]} ")
                    file.write("\n")

                activation_function_type: str = None
                if type(layer.activation_function) is LinearActivationFunction:
                    activation_function_type = "linear"
                elif type(layer.activation_function) is ReLUActivationFunction:
                    activation_function_type = "relu"
                elif type(layer.activation_function) is SigmoidActivationFunction:
                    activation_function_type = "sigmoid"
                elif type(layer.activation_function) is SoftmaxActivationFunction:
                    activation_function_type = "softmax"
                else:
                    raise NotImplementedError()

                file.write(f"{activation_function_type}\n")

    @staticmethod
    def learn_from_file(path: str) -> NeuralNetwork:
        with open(path, "r") as file:
            input_size = int(file.readline().strip())
            num_of_layers = int(file.readline().strip())

            prev_num_of_perceptrons = input_size
            list_of_layer: list[Layer] = []
            latest_activation_function_type: str = None

            for _ in range(num_of_layers):
                num_of_perceptrons = int(file.readline().strip())
                list_of_weight_row: list[Row] = []

                for _ in range(prev_num_of_perceptrons + 1):
                    weight = np.array(
                        list(map(float, file.readline().strip().split()))
                    )
                    row = Row(weight)
                    list_of_weight_row.append(row)

                activation_function_type = file.readline().strip()

                if latest_activation_function_type == "softmax":
                    raise NotImplementedError()

                activation_function: ActivationFunction
                if activation_function_type == "linear":
                    activation_function = LinearActivationFunction()
                elif activation_function_type == "relu":
                    activation_function = ReLUActivationFunction()
                elif activation_function_type == "sigmoid":
                    activation_function = SigmoidActivationFunction()
                elif activation_function_type == "softmax":
                    activation_function = SoftmaxActivationFunction()
                else:
                    raise NotImplementedError()

                layer = Layer(list_of_weight_row, activation_function)
                list_of_layer.append(layer)

                prev_num_of_perceptrons = num_of_perceptrons
                latest_activation_function_type = activation_function_type

            initial_neural_network = NeuralNetwork(list_of_layer)

            test_case_size = int(file.readline().strip())
            input_array: list[np.ndarray] = []
            target_array: list[np.ndarray] = []

            for _ in range(test_case_size):
                input_vector = list(
                    map(float, file.readline().strip().split())
                )

                input_array.append(np.array(input_vector))

            for _ in range(test_case_size):
                target_vector = list(
                    map(float, file.readline().strip().split())
                )
                target_array.append(np.array(target_vector))

            learning_rate = float(file.readline().strip())
            mini_batch_size = int(file.readline().strip())
            max_iter = int(file.readline().strip())
            threshold = float(file.readline().strip())

            backpropagation = Backpropagation(
                initial_neural_network,
                np.array(input_array),
                np.array(target_array)
            )

            error_function: ErrorFunction
            if latest_activation_function_type == "linear" or latest_activation_function_type == "relu" or latest_activation_function_type == "sigmoid":
                error_function = SumOfSquaredErrorFunction()
            elif latest_activation_function_type == "softmax":
                error_function = CrossEntropyErrorFunction()
            else:
                raise NotImplementedError()

            return backpropagation.learn(learning_rate, mini_batch_size, max_iter, threshold, error_function)


### Test Case

#### Linear Small LR

In [18]:
neural_network = FileSystem.learn_from_file(
    "./test-cases/txt/linear_small_lr.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()
        

Epoch 1	|	Error: 0.665	|	Time: 0.0 ms
[[ 0.1012  0.3006  0.1991]
 [ 0.4024  0.201  -0.7019]
 [ 0.1018 -0.799   0.4987]]



#### Linear Two Iteration

In [19]:
neural_network = FileSystem.learn_from_file(
    "./test-cases/txt/linear_two_iteration.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 0.665	|	Time: 0.9999 ms
Epoch 2	|	Error: 0.1818	|	Time: 1.0109 ms
[[ 0.166  0.338  0.153]
 [ 0.502  0.226 -0.789]
 [ 0.214 -0.718  0.427]]



#### Linear

In [20]:
neural_network = FileSystem.learn_from_file(
    "./test-cases/txt/linear.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 0.665	|	Time: 0.0 ms
[[ 0.22  0.36  0.11]
 [ 0.64  0.3  -0.89]
 [ 0.28 -0.7   0.37]]



#### MLP

In [21]:
neural_network = FileSystem.learn_from_file(
    "./test-cases/txt/mlp.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 0.8186	|	Time: 1.0121 ms
[[ 0.07115   0.1403  ]
 [ 0.42885  -0.4403  ]
 [ 0.685575  0.77015 ]]

[[ 0.021     0.1945  ]
 [ 0.39605  -0.500275]
 [ 0.6131    0.79395 ]]



#### RelU

In [22]:
neural_network = FileSystem.learn_from_file(
    "./test-cases/txt/relu.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 0.1463	|	Time: 0.0 ms
[[ 0.105   0.19    0.25  ]
 [ 0.395  -0.49    0.575 ]
 [ 0.7025  0.795  -0.85  ]]



#### Sigmoid Mini Batch

In [23]:
neural_network = FileSystem.learn_from_file(
    "./test-cases/txt/sigmoid_mini_batch_GD.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 1.0857	|	Time: 0.0 ms
Epoch 2	|	Error: 1.0796	|	Time: 0.0 ms
Epoch 3	|	Error: 1.0736	|	Time: 0.0 ms
Epoch 4	|	Error: 1.0676	|	Time: 0.0 ms
Epoch 5	|	Error: 1.0617	|	Time: 1.0049 ms
Epoch 6	|	Error: 1.0558	|	Time: 0.0 ms
Epoch 7	|	Error: 1.05	|	Time: 0.0 ms
Epoch 8	|	Error: 1.0442	|	Time: 1.0021 ms
Epoch 9	|	Error: 1.0385	|	Time: 0.0 ms
Epoch 10	|	Error: 1.0329	|	Time: 0.0 ms
Epoch 11	|	Error: 1.0274	|	Time: 0.9933 ms
Epoch 12	|	Error: 1.022	|	Time: 0.0 ms
Epoch 13	|	Error: 1.0166	|	Time: 0.0 ms
Epoch 14	|	Error: 1.0114	|	Time: 0.9978 ms
Epoch 15	|	Error: 1.0062	|	Time: 0.0 ms
Epoch 16	|	Error: 1.0011	|	Time: 0.0 ms
Epoch 17	|	Error: 0.9962	|	Time: 0.0 ms
Epoch 18	|	Error: 0.9913	|	Time: 1.0114 ms
Epoch 19	|	Error: 0.9865	|	Time: 0.0 ms
Epoch 20	|	Error: 0.9818	|	Time: 0.9918 ms
Epoch 21	|	Error: 0.9772	|	Time: 0.0 ms
Epoch 22	|	Error: 0.9726	|	Time: 0.0 ms
Epoch 23	|	Error: 0.9682	|	Time: 0.9956 ms
Epoch 24	|	Error: 0.9639	|	Time: 0.0 ms
Epoch 25	|	Error: 0.9596	|	Time

#### Sigmoid Stochastic

In [24]:
neural_network = FileSystem.learn_from_file(
    "./test-cases/txt/sigmoid_stochastic_GD.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 1.0857	|	Time: 1.9972 ms
Epoch 2	|	Error: 1.0797	|	Time: 1.0006 ms
Epoch 3	|	Error: 1.0736	|	Time: 0.0 ms
Epoch 4	|	Error: 1.0676	|	Time: 0.9997 ms
Epoch 5	|	Error: 1.0617	|	Time: 1.0092 ms
Epoch 6	|	Error: 1.0558	|	Time: 0.0 ms
Epoch 7	|	Error: 1.05	|	Time: 1.0006 ms
Epoch 8	|	Error: 1.0442	|	Time: 0.0 ms
Epoch 9	|	Error: 1.0385	|	Time: 0.9935 ms
Epoch 10	|	Error: 1.0329	|	Time: 0.0 ms
Epoch 11	|	Error: 1.0274	|	Time: 1.0004 ms
Epoch 12	|	Error: 1.022	|	Time: 0.0 ms
Epoch 13	|	Error: 1.0166	|	Time: 0.9978 ms
Epoch 14	|	Error: 1.0113	|	Time: 0.0 ms
Epoch 15	|	Error: 1.0061	|	Time: 0.9985 ms
Epoch 16	|	Error: 1.001	|	Time: 1.0138 ms
Epoch 17	|	Error: 0.996	|	Time: 0.0 ms
Epoch 18	|	Error: 0.9911	|	Time: 0.993 ms
Epoch 19	|	Error: 0.9863	|	Time: 0.0 ms
Epoch 20	|	Error: 0.9816	|	Time: 0.9997 ms
Epoch 21	|	Error: 0.9769	|	Time: 0.0 ms
Epoch 22	|	Error: 0.9724	|	Time: 0.9966 ms
Epoch 23	|	Error: 0.9679	|	Time: 0.0 ms
Epoch 24	|	Error: 0.9636	|	Time: 0.998 ms
Epoch 25	|	Err

#### Sigmoid

In [25]:
neural_network = FileSystem.learn_from_file(
    "./test-cases/txt/sigmoid.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 0.4585	|	Time: 0.0 ms
[[ 0.10075983  0.19717917]
 [ 0.4        -0.5       ]
 [ 0.70113578  0.79860368]]



#### Softmax Error Only

In [26]:
neural_network = FileSystem.learn_from_file(
    "./test-cases/txt/softmax_error_only.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 0.2204	|	Time: 1.0099 ms
[[ 0.38021839  0.71978161]
 [ 0.21978161 -0.91978161]]



#### Softmax

In [27]:
neural_network = FileSystem.learn_from_file(
    "./test-cases/txt/softmax.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 2.2933	|	Time: 0.0 ms
[[ 0.11301357  0.18698643]
 [ 0.29539055 -0.39539055]
 [ 0.79810267  0.70189733]]



#### SSE Only

In [28]:
neural_network = FileSystem.learn_from_file(
    "./test-cases/txt/sse_only.txt"
)
for weight in neural_network.get_weight():
    print(weight)
    print()

Epoch 1	|	Error: 0.25	|	Time: 0.0 ms
[[0.495 0.505]
 [0.    0.   ]]



#### Learning Dataset Iris

In [30]:
iris = load_iris()
learning_data = np.array(iris.data)
learning_target = np.array(iris.target)

one_hot_learning_target = np.zeros(
    (learning_target.size, learning_target.max() + 1)
)
one_hot_learning_target[np.arange(
    learning_target.size), learning_target] = 1

X_train, X_test, y_train, y_test = train_test_split(
    learning_data,
    one_hot_learning_target,
    test_size=0.2,
    random_state=42
)
numpy_X_train = np.array(X_train)
numpy_X_test = np.array(X_test)
numpy_y_train = np.array(y_train)
numpy_y_test = np.array(y_test)

neural_network = FileSystem.load_from_file(
    "./model/iris-raw.txt"
)
backpropagation = Backpropagation(
    neural_network,
    numpy_X_train,
    numpy_y_train
)

cross_entropy = CrossEntropyErrorFunction()

new_neural_network = backpropagation.learn(
    learning_rate=0.01,
    mini_batch_size=1,
    max_iter=10000,
    threshold=0.05 * len(numpy_X_train),
    error_function=cross_entropy
)
print()

print("Weight:")
for weight in neural_network.get_weight():
    print(weight)
print()

y_pred = new_neural_network.get_batch_output(numpy_X_test)

print(f"Prediction:\n{y_pred}")
print()

print(f"Actual:\n{numpy_y_test}")
print()

error = cross_entropy(y_pred, numpy_y_test)
print(f"Error: {error} / {round(100 * error / len(numpy_X_test), 2)}%")

FileSystem.save_to_file(
    new_neural_network,
    "./model/iris-learn.txt"
)

Epoch 1	|	Error: 131.8335	|	Time: 47.9913 ms
Epoch 2	|	Error: 131.9638	|	Time: 51.9891 ms
Epoch 3	|	Error: 132.0501	|	Time: 46.005 ms
Epoch 4	|	Error: 132.0797	|	Time: 42.0089 ms
Epoch 5	|	Error: 132.0887	|	Time: 44.9948 ms
Epoch 6	|	Error: 132.091	|	Time: 49.0005 ms
Epoch 7	|	Error: 132.0913	|	Time: 63.9987 ms
Epoch 8	|	Error: 132.0911	|	Time: 43.9897 ms
Epoch 9	|	Error: 132.0906	|	Time: 42.001 ms
Epoch 10	|	Error: 132.0901	|	Time: 41.0035 ms
Epoch 11	|	Error: 132.0896	|	Time: 42.9983 ms
Epoch 12	|	Error: 132.089	|	Time: 49.0072 ms
Epoch 13	|	Error: 132.0885	|	Time: 47.0088 ms
Epoch 14	|	Error: 132.0879	|	Time: 40.6067 ms
Epoch 15	|	Error: 132.0874	|	Time: 41.9974 ms
Epoch 16	|	Error: 132.0869	|	Time: 44.0061 ms
Epoch 17	|	Error: 132.0863	|	Time: 49.0067 ms
Epoch 18	|	Error: 132.0858	|	Time: 41.9927 ms
Epoch 19	|	Error: 132.0853	|	Time: 39.9916 ms
Epoch 20	|	Error: 132.0848	|	Time: 53.0081 ms
Epoch 21	|	Error: 132.0842	|	Time: 55.9525 ms
Epoch 22	|	Error: 132.0837	|	Time: 41.9993 ms
E